# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [7]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date (leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(0,"","",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-14T11:47:00 to 2019-02-15T11:47:00
280


In [8]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [9]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
binaryMatrix = fe.encoding(0, df=articleDB)

Binary Encoding


100%|██████████████████████████████████████████████████████████████████████████████| 203/203 [00:00<00:00, 3068.79it/s]


In [10]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(binaryMatrix, articleDB)

                                               title  \
0  UPDATE 3-Russian markets flinch at prospect of...   
1  Real Time Economics: The U.S. Economy Gets a D...   
2  Ten things you probably don't know about the N...   
3  Ikea woos India's rising consumer class, tappi...   
4  UPDATE 1-U.S., India seek to boost trade ties ...   

                                         description  
0  The threat of new U.S. sanctions rocked Russia...  
1  This is the web version of the WSJ’s newslette...  
2  Tech columnist Marc Saltzman lets you in on te...  
3  HYDERABAD, India (AP) — In Hyderabad's bustlin...  
4  The United States and India plan to boost bila...  


In [11]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|████████████████████████████████████████████████████████████████████████████████| 203/203 [00:16<00:00,  5.30it/s]


In [12]:
articleDB[articleDB['Rel'] >= 0.25]

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,tags,tags_top_5
0,0.126395,0.873605,https://www.reuters.com/article/us-russia-mark...,1,UPDATE 3-Russian markets flinch at prospect of...,The threat of new U.S. sanctions rocked Russia...,reuters,2019-02-14T17:05:32Z,MOSCOW LONDON Reuters The threat of new U ...,MOSCOW/LONDON (Reuters) - The threat of new U....,"moscow london, new sanctions, russian markets,...","proposed sanctions,rouble debt,ukraine,dollar ..."
1,0.270946,0.729054,https://blogs.wsj.com/economics/2019/02/15/rea...,1,Real Time Economics: The U.S. Economy Gets a D...,This is the web version of the WSJ’s newslette...,the-wall-street-journal,2019-02-15T10:27:51Z,This is the web version of the WSJs newsletter...,This is the web version of the WSJs newsletter...,"broad agreement, beijings, chinese goods, beij...","evident middle,relief economythe,made cars,chi..."
2,0.371801,0.628199,https://www.usatoday.com/story/tech/columnist/...,1,Ten things you probably don't know about the N...,Tech columnist Marc Saltzman lets you in on te...,usa-today,2019-02-15T11:00:09Z,The Nintendo Switch game console Photo11 Nint...,The Nintendo Switch game console.\r\n (Photo11...,"nintendo switch, nintendo, console snaps, inte...","secondly,maybe youre,press button,also lets,si..."
3,0.380858,0.619142,https://apnews.com/4af42bae385d4a9fb30314814ca...,1,"Ikea woos India's rising consumer class, tappi...","HYDERABAD, India (AP) — In Hyderabad's bustlin...",associated-press,2019-02-15T05:15:00Z,HYDERABAD India AP In Hyderabad s bustling ...,"HYDERABAD, India (AP) In Hyderabad's bustling ...","ikea, america china, given saturation, europe,...","diy,store sits,mumbai,important market,syed na..."
4,0.404275,0.595725,https://www.reuters.com/article/usa-trade-indi...,1,"UPDATE 1-U.S., India seek to boost trade ties ...",The United States and India plan to boost bila...,reuters,2019-02-14T14:46:44Z,Two nations create 7 CEO groups to push bila...,* Two nations create 7 CEO groups to push bila...,"ceo, bilateral trade, chief executives, ceos, ...","chief executives,heinrich,bilateral trade,trad..."
5,0.407504,0.592496,https://www.usatoday.com/story/news/politics/2...,1,Border security bill limits shackling of pregn...,Congress is not only giving Trump money for a ...,usa-today,2019-02-14T20:59:15Z,President Donald Trump says the wall he envisi...,President Donald Trump says the wall he envisi...,"donald trump, southern border, wall way, donal...","drafts legislation,public report,risk poses,pa..."
6,0.414552,0.585448,https://www.washingtonpost.com/us-policy/2019/...,1,‘Here’s the system; it sucks’: Meet the Hill s...,‘Here’s the system; it sucks’: Meet the Hill s...,the-washington-post,2019-02-14T20:06:07Z,One is the daughter of an immigrant from Niger...,One is the daughter of an immigrant from Niger...,"daughter immigrant, tennessee, foe ultrawealth...","mostly activists,mistakes office,riffle,world ..."
7,0.423588,0.576412,https://www.ft.com/content/34f07196-305d-11e9-...,1,US retail sales in December shrink by most sin...,Markets caught off-guard by sharp drop during ...,financial-times,2019-02-14T13:52:31Z,US retail sales unexpectedly fell in December ...,US retail sales unexpectedly fell in December ...,"retail sales, caught markets, cent gain, month...","season numbers,wholesale inflation,upward revi..."
8,0.447926,0.552074,https://www.businessinsider.com/airbus-executi...,1,An Airbus autonomous flight executive explains...,"Airbus, through its innovation labs, is workin...",business-insider,2019-02-14T15:42:24Z,Autonomous flights will be here before we know...,Autonomous flights will be here before we know...,"autonomous, uber, airbus, autonomous aircraft,...","whole system,alabama,network place,sky aim,sha..."
9,0.458842,0.541158,https://apnews.com/19ab0d3b8b154cadaa32aeb18d8...,1,Markets Right Now: US stock are mostly lower a...,NEW YORK (AP) — The latest on developments in ...,associated-press,2019-02-14T17:0

In [13]:
import Recommender as rec
#FinalArticles = rec.recommender(temp, articleDB)

In [14]:
#FinalArticles.to_excel("FinalArticles.xlsx")

In [15]:
import frontPage as fp
fp.FrontPage(articleDB, trendingTermsDB)